In [3]:
import os
import torch
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

In [4]:
path = './Data/sample_test.csv'
out = './Data/sample_demo.csv'

test_df = pd.read_csv(path, sep='\t')
test_df = test_df.dropna()
print(test_df.shape)

(6, 3)


In [4]:
from sentence_transformers import SentenceTransformer, util, SentencesDataset, InputExample, losses
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer("all-MiniLM-L6-v2")
model.load_state_dict(torch.load('q1c_sbert_model.pth'))
model = model.to(device)

In [14]:
def calculate_similarity_score(model, df):
    predicted = []
    for i in range(len(df)):
        emb1 = model.encode(df.iloc[i]['sentence1'], convert_to_tensor=True)
        emb2 = model.encode(df.iloc[i]['sentence2'], convert_to_tensor=True)
        cosine_sim = util.pytorch_cos_sim(emb1, emb2)
        predicted.append(cosine_sim.item() * 5)
    return predicted

predicted_scores = calculate_similarity_score(model, test_df)
test_df['score'] = predicted_scores

In [12]:
test_df.to_csv(out, sep='\t', index=False)